## **PROBLEMA DE OPTIMIZACIÓN DE LA DIETA MILITAR**

### **Descripción:**

El objetivo de este problema es determinar la cantidad óptima de raciones de alimento que deben ser consumidas por un soldado para cumplir con sus requerimientos nutricionales diarios a un costo mínimo. Cada ración de alimento tiene un costo asociado y valores específicos de nutrientes esenciales. Las restricciones están determinadas por los requisitos nutricionales diarios mínimos y máximos establecidos para garantizar el adecuado rendimiento físico y mental del personal militar. Las raciones pueden ser fracciones, un soldado puede comer 1.62 raciones de cualquier alimento.

### **Datos:**

-   **Raciones de Alimento**: Una lista de raciones de alimento disponibles para el consumo, cada una con su respectivo costo por unidad y las cantidades mínimas y máximas que se pueden consumir por día.

-   **Nutrientes**: Una lista de nutrientes esenciales que deben ser consumidos diariamente, cada uno con su cantidad mínima y máxima recomendada.

-   **Valores Nutricionales de las Raciones de Alimento**: Una matriz que indica la cantidad de cada nutriente presente en cada ración de alimento.

### **Objetivo:**

Minimizar el costo total de las raciones de alimento consumidas mientras se cumple con las cantidades recomendadas de cada nutriente.

### **Restricciones:**

1. Las cantidades de raciones de alimento consumidas deben estar entre las cantidades mínimas y máximas definidas para cada ración de alimento.
2. La cantidad total de cada nutriente consumido a partir de todas las raciones de alimento debe estar entre las cantidades mínimas y máximas definidas para cada nutriente.

### **Raciones de Alimento**

|       Alimento       | Costo_Unitario | qmin | qmax |
| :------------------: | :------------: | :--: | :--: |
|     Pollo asado      |      0.84      |  1   |  14  |
|   Pasta con salsa    |      0.78      |  0   |  14  |
|    Tomate fresco     |      0.27      |  1   |  10  |
|    Manzana fresca    |      0.24      |  1   |  9   |
|         Uvas         |      0.32      |  2   |  13  |
| Galletas energéticas |      0.03      |  2   |  15  |
|   Leche descremada   |      0.23      |  2   |  12  |
|  Cereal con frutas   |      0.34      |  0   |  9   |
|      Salchichas      |      0.31      |  1   |  15  |

### **Nutrientes**

|    Nutiente     | qmin | qmax  |
| :-------------: | :--: | :---- |
|    Calorías     | 2000 | 2500  |
|     Calcio      | 800  | 1600  |
|     Hierro      |  10  | 30    |
|   Vitamina A    | 5000 | 50000 |
| Fibra dietética |  25  | 100   |
|  Carbohidratos  |  0   | 300   |
|    Proteína     |  50  | 100   |

### **Valores Nutricionales de las Raciones de Alimento**

|         A/N          | Calorías | Calcio | Hierro | Vit_A  | Fibra dietética | Carbohidratos | Proteína |
| :------------------: | :------: | :----: | :----: | :----: | :-------------: | :-----------: | :------: |
|     Pollo asado      |  277.4   |  21.9  |  1.8   |  77.4  |        0        |       0       |   42.2   |
|   Pasta con salsa    |  358.2   |  80.2  |  2.3   | 3055.2 |      11.6       |     58.3      |   8.2    |
|    Tomate fresco     |   25.8   |  6.2   |  0.6   | 766.3  |       1.4       |      5.7      |    1     |
|    Manzana fresca    |   81.4   |  9.7   |  0.2   |  73.1  |       3.7       |      21       |   0.3    |
|         Uvas         |   15.1   |  3.4   |  0.1   |   24   |       0.2       |      4.1      |   0.2    |
| Galletas energéticas |   78.1   |  6.2   |  0.4   | 101.8  |        0        |      9.3      |   0.9    |
|   Leche descremada   |  121.2   | 296.7  |  0.1   | 500.2  |        0        |     11.7      |   8.1    |
|  Cereal con frutas   |  115.1   |  12.9  |  16.8  | 1250.2 |        4        |     27.9      |    4     |
|      Salchichas      |  242.1   |  23.5  |  2.3   |   0    |        0        |      18       |   10.4   |

### **¿Cómo cambiaría el problema si? (Resolver en notebbok distinto)**

1. Queremos que la ingesta de salchichas represente mínimo el 12% de la ingesta total de alimentos
2. Queremos maximizar la ingesta de proteína como primera prioridad y los costos como segunda


In [1]:
import pandas
from docplex.mp.model import Model
import plotly.graph_objects as go

In [2]:
df_foods = pandas.read_excel("./data.xlsx", sheet_name="Raciones de Alimento", index_col=0)
df_nutrients = pandas.read_excel("./data.xlsx", sheet_name="Nutrientes", index_col=0)
df_food_nutrient = pandas.read_excel("./data.xlsx", sheet_name="Valores Nutricionales", index_col=0)

In [3]:
df_foods

,costo unitario,qmin,qmax
alimento,,,
Pollo asado,0.84,1,14
Pasta con salsa,0.78,0,14
Tomate fresco,0.27,1,10
Manzana fresca,0.24,1,9
Uvas,0.32,2,13
Galletas energéticas,0.03,2,15
Leche descremada,0.23,2,12
Cereal con frutas,0.34,0,9
Salchichas,0.31,1,15


**Conjuntos**


$$ \text{FOODS} : \text{Conjunto de alimentos} $$
$$ \text{NUTRIENTS} : \text{Conjunto de nutrientes} $$


In [4]:
FOODS = df_foods.index.to_list()
NUTRIENTS = df_nutrients.index.to_list()

**Parámetros**


$$ \text{c}_f \in \mathbb{R}^{+}: \text{Costo por unidad del alimento } f \in \text{F} $$
$$ \text{food\_min}_f \in \mathbb{R}^{+}: \text{Cantidad mínima permitida del alimento } f \in \text{F} $$
$$ \text{food\_max}_f \in \mathbb{R}^{+}: \text{Cantidad máxima permitida del alimento } f \in \text{F} $$
$$ \text{nutrient\_min}_n \in \mathbb{R}^{+}: \text{Cantidad mínima requerida del nutriente } n \in \text{N} $$
$$ \text{nutrient\_max}_n \in \mathbb{R}^{+}: \text{Cantidad máxima permitida del nutriente } n \in \text{N} $$
$$ \text{a}_{fn} \in \mathbb{R}^{+}: \text{Cantidad del nutriente } n \in \text{N} \text{ en el alimento } f \in \text{F} $$


In [5]:
## Usaremos los dataframe
print(df_foods.loc["Pollo asado", "costo unitario"])
print(df_foods.loc["Pollo asado", "qmin"])
print(df_foods.loc["Pollo asado", "qmax"])
print(df_nutrients.loc["Calcio", "qmin"])
print(df_nutrients.loc["Calcio", "qmax"])

0.84
1
14
800
1600


**Variables de decisión**

$$ x_f \in \mathbb{R}^{+}: \text{Cantidad del alimento } f \in \text{FOODS} \text{ a consumir} $$
$$ y_n \in \mathbb{R}^{+}: \text{Cantidad del nutriente } n \in \text{NUTRIENTS} \text{ a consumir} $$


In [6]:
# Creación del modelo
model = Model("DietOptimization")

In [7]:
# Variables
x = model.continuous_var_dict(FOODS, name="x")
y = model.continuous_var_dict(NUTRIENTS, name="y")

**Función Objetivo**


$$ \text{minimizar FO} = \sum_{f \in \text{FOODS}} c_f \cdot x_f $$


In [8]:
# Función objetivo: Minimizar el costo total de los alimentos
model.minimize(model.sum(x[f] * df_foods.loc[f, "costo unitario"] for f in FOODS))

**Restricciones**

1. Respetar la cantidad mínima y máxima de cada alimento:
$$ \forall f \in \text{FOODS:} \quad x_f \geq \text{food\_min}_f $$
$$ \forall f \in \text{FOODS:} \quad x_f \leq \text{food\_max}_f $$

2. Satisfacer las necesidades de nutrientes:
$$ \forall n \in \text{NUTRIENTS:} \quad y_n = \sum_{f \in \text{FOODS}} a_{fn} \cdot x_f $$
$$ \forall n \in \text{NUTRIENTS:} \quad y_n \geq \text{nutrient\_min}_n  $$
$$ \forall n \in \text{NUTRIENTS:} \quad y_n \leq \text{nutrient\_max}_n  $$


In [9]:
# 1. Respetar la cantidad mínima y máxima de cada alimento
for f in FOODS:
    model.add_constraint(x[f] >= df_foods.loc[f, "qmin"], ctname=f"food_min_{f}")

for f in FOODS:
    model.add_constraint(x[f] <= df_foods.loc[f, "qmax"], ctname=f"food_max_{f}")

# 2. Satisfacer las necesidades de nutrientes
for n in NUTRIENTS:
    model.add_constraint(y[n] == model.sum(df_food_nutrient.loc[f, n] * x[f] for f in FOODS), ctname=f"rel_{f}_{n}")

for n in NUTRIENTS:
    model.add_constraint(y[n] >= df_nutrients.loc[n, "qmin"], ctname=f"nutrient_min_{n}")

for n in NUTRIENTS:
    model.add_constraint(y[n] <= df_nutrients.loc[n, "qmax"], ctname=f"nutrient_max_{n}")

In [10]:
# Resolviendo el modelo
solution = model.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 32 rows and 2 columns.
Reduced LP has 7 rows, 14 columns, and 62 nonzeros.
Presolve time = 0.00 sec. (0.02 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =             3.182804


In [11]:
solution.objective_value

4.262712702547574

In [12]:
# Imprimiendo la solución
print("FOODS")
for f in FOODS:
    print(f"{f}: {x[f].solution_value}")

print("\nNUTRIENTS")
for n in NUTRIENTS:
    print(f"{n}: {y[n].solution_value}")

FOODS
Pollo asado: 1.0
Pasta con salsa: 1.6810344827586208
Tomate fresco: 1.0
Manzana fresca: 1.0
Uvas: 2.0
Galletas energéticas: 6.383526866528326
Leche descremada: 2.0
Cereal con frutas: 0
Salchichas: 1.0

NUTRIENTS
Calorías: 2000.0
Calcio: 835.896832089717
Hierro: 11.719790056956159
Vitamina A: 7750.939586736721
Fibra: 25.0
Carbohidratos: 233.67111020354102
Proteína: 90.02965693849619


In [13]:
fig = go.Figure()

fig.add_trace(go.Bar(x=FOODS, y=[x[f].solution_value for f in FOODS], text=[f"{x[f].solution_value:.2f}" for f in FOODS],))

for idx, f in enumerate(FOODS):
    fig.add_shape(
        type="line",
        x0=idx - 0.4,
        x1=idx + 0.4,
        y0=df_foods.loc[f, "qmin"],
        y1=df_foods.loc[f, "qmin"],
        line=dict(color="blue", width=2, dash="dash"),
    )
    
    fig.add_shape(
        type="line",
        x0=idx - 0.4,
        x1=idx + 0.4,
        y0=df_foods.loc[f, "qmax"],
        y1=df_foods.loc[f, "qmax"],
        line=dict(color="blue", width=2, dash="dash"),
    )
    
fig.update_layout(
    title="CONSUMO DE RACIONES DE ALIMENTOS",
    xaxis_title="Alimentos",
    yaxis_title="Raciones",
    xaxis={"type": "category"},
    showlegend=False,
    width=800,
    template="ggplot2",
)


fig.show()

In [14]:
fig = go.Figure()

nutrients_percentages = [(y[n].solution_value) / (df_nutrients.loc[n, "qmax"]) for n in NUTRIENTS]
fig.add_trace(
    go.Bar(
        x=NUTRIENTS,
        y=nutrients_percentages,
        text=[f"{p:.2%}" for p in nutrients_percentages],
        textposition="outside",
        marker_color="goldenrod"
    )
)

for idx, n in enumerate(NUTRIENTS):
    fig.add_shape(
        type="line",
        x0=idx - 0.4,
        x1=idx + 0.4,
        y0=df_nutrients.loc[n, "qmin"] / (df_nutrients.loc[n, "qmax"]),
        y1=df_nutrients.loc[n, "qmin"] / (df_nutrients.loc[n, "qmax"]),
        line=dict(color="blue", width=2, dash="dash"),
    )


# Configuraciones adicionales de la gráfica
fig.update_layout(
    title="POECENTAJES DE CUMPLIMIENTO NUTRIENTES",
    xaxis_title="Nutrientes",
    yaxis_title="Porcentaje ",
    xaxis={"type": "category"},
    showlegend=False,
    width=800,
    height=500,
    template="ggplot2",
)

# Mostrar la gráfica
fig.show()

| Concepto  | Descripción                                                 | Ecuación                                                                                      | Código Python                                                                                                   |
| :-------: | ----------------------------------------------------------- | --------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------- |
|  **SET**  | Conjunto de alimentos                                       | $$ \text{FOODS} $$                                                                            | `FOODS = [...]`                                                                                                 |
|  **SET**  | Conjunto de nutrientes                                      | $$ \text{NUTRIENTS} $$                                                                        | `NUTRIENTS = [...]`                                                                                             |
| **PARAM** | Costo por unidad del alimento \(f\)                         | $$ \text{c}_f \in \mathbb{R}^{+} $$                                                          | `NUTRIENTS = [...]`                                                                                             |
| **PARAM** | Cantidad mínima permitida del alimento \(f\)                | $$ \text{food\_min}_f \in \mathbb{R}^{+} $$                                                   | `df_foods.loc[f, "qmin"]`                                                                                       |
| **PARAM** | Cantidad máxima permitida del alimento \(f\)                | $$ \text{food\_max}_f \in \mathbb{R}^{+} $$                                                   | `df_foods.loc[f, "qmax"]`                                                                                       |
| **PARAM** | Cantidad mínima requerida del nutriente \(n\)               | $$ \text{nutrient\_min}_n \in \mathbb{R}^{+} $$                                               | `df_nutrients.loc[n, "qmin"]`                                                                                   |
| **PARAM** | Cantidad máxima permitida del nutriente \(n\)               | $$ \text{nutrient\_max}_n \in \mathbb{R}^{+} $$                                               | `df_nutrients.loc[n, "qmax"]`                                                                                   |
| **PARAM** | Cantidad del nutriente \(n\) en el alimento \(f\)           | $$ \text{a}_{fn} \in \mathbb{R}^{+} $$                                                       | `df_food_nutrient.loc[f, n]`                                                                                    |
|  **VAR**  | Variables de alimentos                                      | $$ x_f \in \mathbb{R}^{+} $$                                                                  | `x = model.continuous_var_dict(FOODS, name="x")`                                                                |
|  **VAR**  | Variables de nutrientes                                     | $$ y_n \in \mathbb{R}^{+} $$                                                                  | `y = model.continuous_var_dict(NUTRIENTS, name="y")`                                                            |
|  **FO**   | Función objetivo: Minimizar el costo total de los alimentos | $$ \text{minimizar FO} = \sum_{f \in \text{FOODS}} c_f \cdot x_f $$                          | `model.minimize(model.sum(x[f] * df_foods.loc[f, "costo unitario"] for f in FOODS))`                            |
| **CONST** | Restricción mínima de alimentos                             | $$ \forall f \in \text{FOODS:} \quad x_f \geq \text{food\_min}_f $$                           | `model.add_constraints(x[f] >= df_foods.loc[f, "qmin"] for f in FOODS)`                                         |
| **CONST** | Restricción máxima de alimentos                             | $$ \forall f \in \text{FOODS:} \quad x_f \leq \text{food\_max}_f $$                           | `model.add_constraints(x[f] <= df_foods.loc[f, "qmax"] for f in FOODS)`                                         |
| **CONST** | Relación nutrientes-alimentos                               | $$ \forall n \in \text{NUTRIENTS:} \quad y_n = \sum_{f \in \text{FOODS}} a_{fn} \cdot x_f $$ | `model.add_constraints(y[n] == model.sum(df_food_nutrient.loc[f, n] * x[f] for f in FOODS) for n in NUTRIENTS)` |
| **CONST** | Restricción mínima de nutrientes                            | $$ \forall n \in \text{NUTRIENTS:} \quad y_n \geq \text{nutrient\_min}_n $$                   | `model.add_constraints(y[n] >= df_nutrients.loc[n, "qmin"] for n in NUTRIENTS)`                                 |
| **CONST** | Restricción máxima de nutrientes                            | $$ \forall n \in \text{NUTRIENTS:} \quad y_n \leq \text{nutrient\_max}_n $$                    | `model.add_constraints(y[n] <= df_nutrients.loc[n, "qmax"] for n in NUTRIENTS)`                                 |
